# 7. Interactive visualization


In this module, we're going to cover interactive visualization with [leafmap](https://leafmap.org/).

The end result will be similar to GIS tools like ArcGIS. But by having the visualization in python you get some big benefits compared to GIS

* Programatic nature means you can tweak your analaysis/data, and rebuild the visualization instantly.
* You get access to all python's tools, more than GIS toolboxes.
* Because your viz is code, it can be repeated, version controlled, forked.

We'll cover vizualising each of the different "cardinalities" of geospatial data

* Points
* Lines
* Polygons
* Rasters


## Creating a map with leafmap

Leafmap is a python library that contains tools for creating an interactive map-based visualisation, as well as a wide variety of tools for downloading and processing geospatial data.

Most everything in leafmap starts by creating a new `Map` instance. To view the map in your notebook, we put just the variable of our map `m` on the last line of the cell: remember, notebooks auto-display their final line


In [1]:
import leafmap

m = leafmap.Map()
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

By default, almost the entire world is shown. We can aim the map at our area of interest using the `center` and `zoom` arguments.

In [ ]:
latlon_carmel = (36.5506, -121.9220)
m = leafmap.Map(
    center=latlon_carmel,
    zoom=13,
)
m

Zooming can be disabled by setting `min_zoom` and `max_zoom`. Panning can be disabled with `dragging`. And leafmap's tools can be hidden with  the `clear_controls()` method.

Here's a static map:

In [ ]:
latlon_carmel = (36.5506, -121.9220)
zoom = 13
m = leafmap.Map(
    center=latlon_carmel,
    zoom=zoom,
    min_zoom=zoom,
    max_zoom=zoom,
    dragging=False,
)
m.clear_controls()
m

## Basemap selection

By default, leafmap uses an OpenStreetMap basemap. But for some applications you want other basemaps.

Leafmap comes with a number of basemaps, an up-to-date list can be found [here](https://xyzservices.readthedocs.io/en/stable/gallery.html).

Here are some basemaps that are particularly useful for data science visualization:

`Esri.WorldTopoMap` feaures terrain shading and contours, which gives context for non-urban areas

In [ ]:
m = leafmap.Map(
    center=latlon_carmel,
    zoom=13,
    basemap="Esri.WorldTopoMap",
)
m

`CartoDB.Positron` as a simple map that won't clash with your visualization overlays (there's also `CartoDB.PositronNoLabels` and `CartoDB.DarkMatter`, can you guess what they look like?)

In [ ]:
m = leafmap.Map(
    center=latlon_carmel,
    zoom=13,
    basemap="CartoDB.Positron",
)
m

`Esri.WorldImagery` has aerial imagery

In [ ]:
m = leafmap.Map(
    center=latlon_carmel,
    zoom=13,
    basemap="Esri.WorldImagery",
)
m

It can be useful to switch between base maps: for examply, you may want a simple layer for data visualisation, but also switch to aerial imagery to orient yourself.

Additional basemaps can be added with the `add_basemap()` method. The `add_layer_control()` method adds a popup to toggle layer visibility.

To keep our plots simple, we're also hiding the `draw` and `toolbar` controls here.

In [ ]:
m = leafmap.Map(
    center=latlon_carmel,
    zoom=13,
    basemap="Esri.WorldImagery",
    draw_control=False,
    toolbar_control=False,
)
m.add_basemap("CartoDB.PositronNoLabels")
m.add_layer_control()
m

## Loading data 

Now we have our base map ready, lets load some data to visualize!

We'll load data from two different sources:

* Streamgage data from a SQL database
* tremaline data from a vector (geojson) file.



### Loading data from SQL


`data/USGSStreamgages.sqlite` is a database file that contains a table `SWIM_gage_loc` with [US streamgage information](https://www.sciencebase.gov/catalog/item/5ebe92af82ce476925e44b8f). 

For this course we're using a [sqlite](https://en.wikipedia.org/wiki/SQLite) database, as no server or authentication is needed: just a URL or file path. But the principles will be the same for other databases like Snowflake and PostGIS: you'll just provide your server hostname and credentials to connect instead.

The pandas function `read_sql_table()` loads all columns and rows from a database into a `DataFrame` object.





In [2]:
import pandas as pd
from sqlalchemy import create_engine


# Connect to database.
#
# This will vary depending on the database being used.
db_connection_string = "sqlite:///data/USGSStreamgages.sqlite"
db_connection = create_engine(db_connection_string)

# Load the entire table.
#
# This will be the same for any SQL database, just provide a database connection object,
# pandas will handle the rest.
df_gage_full = pd.read_sql_table(table_name="SWIM_gage_loc", con=db_connection)

df_gage_full.head()

,index,Gage_no,Gage_name,Gage_type,Lat_nwis,Long_nwis,SqKm_nwis,Lat_snap,Long_snap,COMID,...,State,Pass,CHECKED,QC_notes1,REVIEWED,Rev_notes2,Ver_notes3,Gage_info,QC_final,MovedCOMID
0,0,01098530,"SUDBURY RIVER AT SAXONVILLE, MA",ContGage,42.325373,-71.397560,274.538739,42.325326,-71.397449,6772927,...,MA,1,None,None,None,None,None,None,AutoRef,None
1,1,01122680,"MERRICK BK NR SCOTLAND, CT.",CrestStage,41.728988,-72.085076,13.493838,41.728987,-72.084926,6162963,...,CT,1,None,None,None,None,None,None,AutoRef,None
2,2,01176000,"QUABOAG RIVER AT WEST BRIMFIELD, MA",ContGage,42.182316,-72.263691,388.498215,42.182412,-72.263445,7690649,...,MA,1,None,None,None,None,None,None,AutoRef,None
3,3,01173500,"WARE RIVER AT GIBBS CROSSING, MA",ContGage,42.236204,-72.272580,510.227656,42.236129,-72.272568,7690465,...,MA,1,None,None,None,None,None,None,AutoRef,None
4,4,01127500,"YANTIC RIVER AT YANTIC, CT",ContGage,41.558709,-72.121467,231.285937,41.558570,-72.121492,6168318,...,CT,1,"JAF, 12/21/2015",OK.,"GCB, 3/24/2017","Yes agree with move, location of streamgage is...",None,None,OK,None


That's a lot of columns! A tip for viewing a dataframe that's so wide that has too many columns for the notebook to display them all, is to take the transpose of the first few rows.

Transposing a 2D data structure means swapping the rows and columns. Here's a numpy array:

In [3]:
import numpy as np

array = np.array(
    [
        [1, 2, 3, 4, 5],
        [11, 12, 13, 14, 15],
    ]
)

print(array)

[[ 1  2  3  4  5]
 [11 12 13 14 15]]


In [ ]:
print(array.transpose())

Performing the same operation on a pandas data frame "rotates" our data, so instead of being "wide" (which often means columns get truncated) it becomes "tall" (which allows us to scroll through the data.)

In [ ]:
df_gage_full.head(2).transpose()

Lets extract just a subset of this data. We could of course use our pandas skills to filter out the relevant rows and columns! But this kind of simple filtering is exactly what SQL is designed for, and should load much faster as  we're not requesting unneeded data from the database.

Can you guess what our dataframe will look like after running this query?


In [5]:
query = """
SELECT
    Gage_no,
    Gage_name,
    Lat_nwis,
    Long_nwis,
    Pass
FROM SWIM_gage_loc
WHERE State = 'CA'
"""

df_gage = pd.read_sql_query(query, db_connection)
df_gage.head()

,Gage_no,Gage_name,Lat_nwis,Long_nwis,Pass
0,09429130,"P.V.I.D. OLIVE LAKE DRAIN NEAR BLYTHE, CA",33.683912,-114.537734,0
1,09526200,"YPSILANTI CANAL NEAR WINTERHAVEN, CA",32.768379,-114.655509,0
2,09530400,"RESERVATION DRAIN NO. 11 NR WINTERHAVEN, CA",32.756435,-114.686066,0
3,09429200,"PVID C CANAL SPILL NEAR BLYTHE, CA",33.427807,-114.673291,0
4,09525500,YUMA MAIN CANAL BLW COLORADO R. SIPHON AT YUMA...,32.730325,-114.619952,1


Finally, because we have geospatial information in the dataframe, let's convert it from a `pandas` to a `geopandas` data frame, to give us access to the handy geospatial methods for processing the point locations.

In [6]:
import geopandas as gpd

gdf_gage = gpd.GeoDataFrame(df_gage, geometry=gpd.points_from_xy(df_gage.Long_nwis, df_gage.Lat_nwis), crs="EPSG:4326")

gdf_gage.head()

,Gage_no,Gage_name,Lat_nwis,Long_nwis,Pass,geometry
0,09429130,"P.V.I.D. OLIVE LAKE DRAIN NEAR BLYTHE, CA",33.683912,-114.537734,0,POINT (-114.53773 33.68391)
1,09526200,"YPSILANTI CANAL NEAR WINTERHAVEN, CA",32.768379,-114.655509,0,POINT (-114.65551 32.76838)
2,09530400,"RESERVATION DRAIN NO. 11 NR WINTERHAVEN, CA",32.756435,-114.686066,0,POINT (-114.68607 32.75644)
3,09429200,"PVID C CANAL SPILL NEAR BLYTHE, CA",33.427807,-114.673291,0,POINT (-114.67329 33.42781)
4,09525500,YUMA MAIN CANAL BLW COLORADO R. SIPHON AT YUMA...,32.730325,-114.619952,1,POINT (-114.61995 32.73032)


### Loading data from Snowflake

Snowflake is a SQL database, so all of the above queries will work on Snowflake too!

The main difference is in building the snowflake connection. Snowflake isn't supported by pandas by default, but Snowflake provides a few different packages to help with that.

The easiest is to install Snowflake's `snowflake-sqlalchemy` package

```bash
pip install snowflake-sqlalchemy
```

Now we can use the same approach as for SQLite, but instead of the filepath to the SQLite database, we use the Snowflake connection format: `snowflake://<username>:<password>@<account>/<database>`

```python
import pandas as pd
from sqlalchemy import create_engine


# Connect to database.
password = read_snowflake_password()
db_connection_string = "snowflake://example-username:" + password + "@example-organization/USGSStreamgages"
db_connection = create_engine(db_connection_string)

# Load the entire table.
#
# This will be the same for any SQL database, just provide a database connection object,
# pandas will handle the rest.
df_gage_full = pd.read_sql_table(table_name="SWIM_gage_loc", con=db_connection)
```

Snowflake also comes with a helper to build this the connection URL for you:

```python
from snowflake.sqlalchemy import URL

db_connection_string = URL(
    account = "example-organization",
    user = "example-username",
    password = read_snowflake_password(),
    database = "USGSStreamgages",
)
```


Note that in the above examples, we use a function to load the password, rather than putting the password in the code (where it could easily be accidentally uploaded to GitHub or shared with others!). 



### Loading vector data

Geopandas can read most vector file directly, including shapefiles and geojson:

In [7]:
gdf_streamlines = gpd.read_file("./data/California_Streams.SanFranciscoBay.geojson")
gdf_streamlines.head()

,OBJECTID,Name,DFGWATERID,Mouth_Lat,Mouth_Long,Down_Name,Down_ID,Mouth_Meas,Source_Meas,EditVersion,...,GNIS_ID,NHD_Permanent_Identifier,Alternate_Name,Enabled,NHD_Edit,Hydrologic_Subregion,Hydrologic_Subbasin,Hydrologic_Subbasin_Code,Shape__Length,geometry
0,357044,None,30935419,37.259144,-122.402551,Butano Creek,30935567,0.0,717.112662,NaN,...,None,30935419,None,1.0,None,SanFranciscoBay,San Francisco Coastal South,18050006,901.824740,"LINESTRING (-122.40434 37.26203, -122.40448 37..."
1,357045,None,30935241,37.286834,-122.407599,,0,0.0,2179.301023,NaN,...,None,30935241,None,1.0,None,SanFranciscoBay,San Francisco Coastal South,18050006,2741.302394,"LINESTRING (-122.38997 37.28735, -122.3901 37...."
2,357046,None,30935229,37.291405,-122.406511,,0,0.0,1733.418379,NaN,...,None,30935229,None,1.0,None,SanFranciscoBay,San Francisco Coastal South,18050006,2179.955449,"LINESTRING (-122.38814 37.29293, -122.38825 37..."
3,357047,None,30935105,37.368074,-122.408480,,0,0.0,1569.742449,NaN,...,None,30935105,None,1.0,None,SanFranciscoBay,San Francisco Coastal South,18050006,1976.878604,"LINESTRING (-122.39648 37.37373, -122.39666 37..."
4,357049,None,30931433,37.387416,-122.418851,,0,0.0,2114.012058,NaN,...,None,30931433,None,1.0,None,SanFranciscoBay,San Francisco Coastal South,18050006,2663.465437,"LINESTRING (-122.40569 37.39883, -122.40585 37..."


## Displaying point data

Leafmap provides `add_markers_from_xy()` to add points from a dataframe.



In [ ]:
latlon_fresno = 36.857, -119.786
m = leafmap.Map(
    center=latlon_fresno,
    zoom=6,
    basemap="Esri.WorldTopoMap",
    draw_control=False,
    toolbar_control=False,
)

m.add_circle_markers_from_xy(
    data=gdf_gage,
)


m

Try clicking on one of the points: you get a popup showing the value of the other attributes of that streamgage.

With so many points, it's a bit hard to make sense of the data. Leafmap gives you [options](https://leafmap.org/leafmap/?h=add_circle_markers_from_xy#leafmap.leafmap.Map.add_circle_markers_from_xy) to style your markers: lets decrease the radius and opacity of the markers to get a better sense of the density.

As well as passing a constant value like `opacity=0.5`, we can also pass a list of values representing a different value for each marker. We'll also color-code the markers individually based on the "Pass" column of our dataframe (whether the streamgage data passed a QA check):


In [18]:
latlon_fresno = 36.857, -119.786
m = leafmap.Map(
    center=latlon_fresno,
    zoom=6,
    basemap="Esri.WorldTopoMap",
    draw_control=False,
    toolbar_control=False,
)

color = ["blue" if passed else "gray" for passed in gdf_gage.Pass]

m.add_circle_markers_from_xy(
    data=gdf_gage,
    radius=2,  # Default is 10.
    opacity=0.5,  # Default is 1.
    color=color,
)

m

Map(center=[36.857, -119.786], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…



Reducing the marker size and opacity makes the map fairly readable. But for larger numbers, both visibility and browsing performance will start to suffer!

For even larger numbers of points, leafmap provides `add_marker_cluster()`. Each cluster displays how many points it includes: zooming in breaks up the clusters. 

In [ ]:
latlon_fresno = 36.857, -119.786
m = leafmap.Map(
    center=latlon_fresno,
    zoom=6,
    basemap="Esri.WorldTopoMap",
    draw_control=False,
    toolbar_control=False,
)

m.add_marker_cluster(
    data=gdf_gage,
    x="Long_nwis",  # Must provide the names of the x/y columns.
    y="Lat_nwis",
)

m

## Displaying linear data

Leafmap provides the `add_gdf()` function that can plot geopandas `GeoDataFrame` objects, including those with line and polygon geometry.

Here we load some streamlines around the SF bay area.

In [ ]:
latlon_sf = 37.715, -122.475
m = leafmap.Map(
    center=latlon_sf,
    zoom=10,
    basemap="CartoDB.DarkMatter",
    draw_control=False,
    toolbar_control=False,
)

# Color the lines.
subbasin_colors = {
    "Tomales-Drake Bays": "blue",
    "San Francisco Coastal South": "purple",
}
style_callback = lambda feat: {"color": subbasin_colors[feat["properties"]["Hydrologic_Subbasin"]], "weight": 1}

# Plot the lines.
m.add_gdf(
    gdf_streamlines,
    style_callback=style_callback,
)

# Add a legend.
m.add_legend(title="Sub Basin", legend_dict=subbasin_colors)

m

## Displaying polygon data

Leafmap also comes with a number of helpful tools for downloading data.

The `get_wbd()` function returns watershed polygons based on a spatial query.

In [ ]:
m = leafmap.Map(
    center=latlon_carmel,
    zoom=12,
    basemap="Esri.WorldImagery",
    draw_control=False,
    toolbar_control=False,
)

point_geometry = {"y": latlon_carmel[0], "x": latlon_carmel[1]}
gdf_huc12 = leafmap.get_wbd(point_geometry, digit=12, return_geometry=True)
m.add_gdf(
    gdf_huc12,
)

m

Geopandas has the concept of a [spatial join](https://geopandas.org/en/stable/gallery/spatial_joins.html), using the `sjoin()` method. It lets you merge GeoDataFrame objects be finding rows whose geometry intersects.

For example: we can use a spatial join take our streamgage data frame, and for each gage row,  find the watershed that contains the point, and add the watershed attributes to the gage row.

In [33]:
# Get HUC4 watershed boundaries for the area containing our streamgages.
xmin, ymin, xmax, ymax = gdf_gage.total_bounds
bbox_geometry = {"xmin": xmin, "ymin": ymin, "xmax": xmax, "ymax": ymax}
gdf_huc2 = leafmap.get_wbd(bbox_geometry, digit=2, return_geometry=True)
gdf_huc2 = gdf_huc2.to_crs(gdf_gage.crs)

# Spatial join with gages.
gdf_gage_watershed = gdf_gage.sjoin(gdf_huc2, how="left", predicate="within")

# Base map.
m = leafmap.Map(
    center=latlon_fresno,
    zoom=7,
    basemap="Esri.WorldImagery",
    draw_control=False,
    toolbar_control=False,
)

# Add the watershed boundaries as the 1st layer.
m.add_gdf(
    gdf_huc2,
    layer_name="watershed",
    style={"opacity": 0.5},
)

# Add our points on top.
m.add_circle_markers_from_xy(
    data=gdf_gage_watershed,
    color="magenta",
    layer_name="gage",
    radius=3,  # Default is 10.
    opacity=0.5,  # Default is 1.
)


m

Map(center=[36.857, -119.786], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

Now the tooltip for each point contains not only the information about the gage, but also the attributes of the containing watershed.

## Displaying raster data

Raster data can be displayed with the `add_raster()` method.

The [viridis](https://cran.r-project.org/web/packages/viridis/vignettes/intro-to-viridis.html) colormap uses hue (color) to communicate more information than grayscale, and was engineered to be easier to read by people with most form of color blindness and color deficiency.


```python
m = leafmap.Map()

m.add_raster(
    "https://www.py4wrds.com/ETOPO1_Ice_g_geotiff.resampled-1deg.cog.tif",
    colormap="viridis",
)
```

![global dem](img/7-raster-map.png)

## Exporting visualizations

Leafmap provides the `to_html()` method for saving maps to disk. The resulting html file can be shared to others and opened in their browser, even without having python installed.

In [ ]:
m = leafmap.Map(
    center=latlon_carmel,
    zoom=13,
    basemap="Esri.WorldTopoMap",
)
m.to_html("./exported_map_analysis.html")